In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .appName('SparkSql')
         .getOrCreate())

In [84]:
file =  '../Rural_Atlas_Update20/People.csv'
columns = !powershell cat {file} -First 1
for line in columns[0].split(','):
    print(line)

FIPS
State
County
PopChangeRate1718
PopChangeRate1018
TotalPopEst2018
NetMigrationRate1018
NaturalChangeRate1018
Net_International_Migration_Rate_2010_2018
PopChangeRate0010
NetMigrationRate0010
NaturalChangeRate0010
Immigration_Rate_2000_2010
PopDensity2010
Under18Pct2010
Age65AndOlderPct2010
WhiteNonHispanicPct2010
BlackNonHispanicPct2010
AsianNonHispanicPct2010
NativeAmericanNonHispanicPct2010
HispanicPct2010
MultipleRacePct2010
NonHispanicWhitePopChangeRate0010
NonHispanicBlackPopChangeRate0010
NonHispanicAsianPopChangeRate0010
NonHispanicNativeAmericanPopChangeRate0010
HispanicPopChangeRate0010
MultipleRacePopChangeRate0010
WhiteNonHispanicNum2010
BlackNonHispanicNum2010
AsianNonHispanicNum2010
NativeAmericanNonHispanicNum2010
HispanicNum2010
MultipleRaceNum2010
ForeignBornPct
ForeignBornEuropePct
ForeignBornMexPct
NonEnglishHHPct
Ed1LessThanHSPct
Ed2HSDiplomaOnlyPct
Ed3SomeCollegePct
Ed4AssocDegreePct
Ed5CollegePlusPct
AvgHHSize
FemaleHHPct
HH65PlusAlonePct
OwnHomePct
Ed5CollegeP

In [39]:
people_df = spark.read.csv(file, inferSchema=True, header=True)
#people_df.show(1)
pop_df_2010 = people_df.select(
    'state', 'county','TotalPopEst2010','WhiteNonHispanicPct2010','HispanicPct2010', 'AsianNonHispanicPct2010')
pop_df_2010.show(5)

+-----+-------------+---------------+-----------------------+---------------+-----------------------+
|state|       county|TotalPopEst2010|WhiteNonHispanicPct2010|HispanicPct2010|AsianNonHispanicPct2010|
+-----+-------------+---------------+-----------------------+---------------+-----------------------+
|   US|United States|      309338421|                  63.75|          16.35|                   4.69|
|   AL|      Alabama|        4785448|                  67.04|           3.88|                   1.11|
|   AL|      Autauga|          54754|                  77.25|            2.4|                   0.86|
|   AL|      Baldwin|         183111|                   83.5|           4.38|                   0.74|
|   AL|      Barbour|          27330|                  46.75|           5.05|                   0.39|
+-----+-------------+---------------+-----------------------+---------------+-----------------------+
only showing top 5 rows



# SaveAsTable and load in sql 

In [41]:
pop_df_2010. write.saveAsTable('2010population', mode='overwrite')

In [43]:
sql_df = spark.sql('select * from 2010population')
sql_df.show(5)

+-----+-------------+---------------+-----------------------+---------------+-----------------------+
|state|       county|TotalPopEst2010|WhiteNonHispanicPct2010|HispanicPct2010|AsianNonHispanicPct2010|
+-----+-------------+---------------+-----------------------+---------------+-----------------------+
|   US|United States|      309338421|                  63.75|          16.35|                   4.69|
|   AL|      Alabama|        4785448|                  67.04|           3.88|                   1.11|
|   AL|      Autauga|          54754|                  77.25|            2.4|                   0.86|
|   AL|      Baldwin|         183111|                   83.5|           4.38|                   0.74|
|   AL|      Barbour|          27330|                  46.75|           5.05|                   0.39|
+-----+-------------+---------------+-----------------------+---------------+-----------------------+
only showing top 5 rows



# Temporary View

In [44]:
pop_df_2010.createOrReplaceTempView('pop2010_df_temp_view')

### FInd counties with more hispanic people than white

In [47]:
spark.sql(' SELECT state, \
          county, \
          WhiteNonHispanicPct2010,  \
          HispanicPct2010 ,\
          HispanicPct2010/TotalPopEst2010 AS hispanic_ratio \
          FROM pop2010_df_temp_view \
          WHERE HispanicPct2010>WhiteNonHispanicPct2010 \
          ORDER BY hispanic_ratio DESC').show(10)

+-----+---------+-----------------------+---------------+--------------------+
|state|   county|WhiteNonHispanicPct2010|HispanicPct2010|      hispanic_ratio|
+-----+---------+-----------------------+---------------+--------------------+
|   TX|   Kenedy|                  20.67|          76.68| 0.18388489208633096|
|   PR|  Culebra|                    7.7|          91.75| 0.05041208791208791|
|   TX|Culberson|                  21.02|          76.19|0.031693011647254576|
|   TX|  Edwards|                   47.3|           51.3|0.025662831415707854|
|   TX| Hudspeth|                  18.07|          79.63|0.022934907834101383|
|   CO| Costilla|                  30.82|          66.03|0.018710682913006517|
|   TX|   Reagan|                   36.2|          60.91|0.018165821652251713|
|   TX| Jim Hogg|                    6.3|          92.58|0.017507564296520423|
|   TX| Crockett|                  35.28|          63.24| 0.01708727370980816|
|   NM|Guadalupe|                  16.07|          7

### Find states with more hispanic people than white in 2010

#### pyspark

In [68]:
from pyspark.sql.functions import *

In [79]:
(sql_df.select('state', 'county', 'TotalPopEst2010',
              (col('TotalPopEst2010')*col('HispanicPct2010')/100).alias('TotalHispanic') ,
             (col('TotalPopEst2010')*col('WhiteNonHispanicPct2010')/100).alias('TotalWhiteNonHispanic'))
 .groupBy('state').agg(
     sum('TotalPopEst2010').alias('TotalPopEst2010'),
     sum('TotalHispanic').alias('TotalHispanic'),
     sum('TotalWhiteNonHispanic').alias('TotalWhiteNonHispanic'),
     )
 .select('state','TotalPopEst2010',
         (col('TotalHispanic')/col('TotalPopEst2010')*100).alias('HispanicPct'),
        (col('TotalWhiteNonHispanic')/col('TotalPopEst2010')*100).alias('WhiteNonHispanicPct'))
 .where(col('HispanicPct')> col('WhiteNonHispanicPct'))
 .sort('HispanicPct',ascending=False)
).show(5)

+-----+---------------+-----------------+-------------------+
|state|TotalPopEst2010|      HispanicPct|WhiteNonHispanicPct|
+-----+---------------+-----------------+-------------------+
|   PR|        7443050|98.99912663088385|   0.72163327399386|
|   NM|        4129176|46.29742528775718|  40.49270082941487|
+-----+---------------+-----------------+-------------------+



#### SQL

In [83]:
spark.sql(
    'select state, \
sum(TotalPopEst2010) as TotalPopEst2010, \
    sum(totalhispanic) / sum(TotalPopEst2010) * 100 as hispanic_pct, \
    sum(totalwhite) / sum(TotalPopEst2010) * 100 as white_pct \
    from \
        (select state, TotalPopEst2010, \
        TotalPopEst2010 * HispanicPct2010 / 100 as totalhispanic, \
        TotalPopEst2010 * WhiteNonHispanicPct2010 / 100 as totalwhite \
        from pop2010_df_temp_view) \
    group by state \
    having hispanic_pct > white_pct' 
).show()

+-----+---------------+-----------------+-----------------+
|state|TotalPopEst2010|     hispanic_pct|        white_pct|
+-----+---------------+-----------------+-----------------+
|   NM|        4129176|46.29742528775718|40.49270082941487|
|   PR|        7443050|98.99912663088385| 0.72163327399386|
+-----+---------------+-----------------+-----------------+



# Create DB

In [88]:
spark.sql('CREATE DATABASE people_db')
spark.sql('USE people_db')

DataFrame[]

# Managed and unmanaged Tables

In [86]:
hh_df= people_df.select('AvgHHSize',
'FemaleHHPct',
'HH65PlusAlonePct',
'OwnHomePct',
'TotalHH',
'NonEnglishHHPct')
hh_df.show(5)


+---------+-----------+----------------+-----------+---------+---------------+
|AvgHHSize|FemaleHHPct|HH65PlusAlonePct| OwnHomePct|  TotalHH|NonEnglishHHPct|
+---------+-----------+----------------+-----------+---------+---------------+
|     2.63|12.70110164|     10.52845279|63.81868061|118825921|    4.464884392|
|     2.55|14.77684811|     11.20841064|68.57437544|  1856695|    1.149407953|
|     2.59|11.40875843|     11.38026028| 73.2877363|    21054|    0.750451221|
|     2.63|9.426923936|     12.84594066|72.85933826|    76133|    0.901054733|
|     2.54|19.36677184|     13.85050593|62.50680013|     9191|    0.816015668|
+---------+-----------+----------------+-----------+---------+---------------+
only showing top 5 rows



## Managed Table
File path is decided by sparksession

### Using python

In [91]:
hh_df.write.saveAsTable('hh_df_from_python')

### Using sql

In [96]:
spark.sql('create table if not exists hh_df_from_sql \
(AvgHHSize INT, FemaleHHPct FLOAT, HH65PlusAlonePct FLOAT, OwnHomePct FLOAT, TotalHH INT, NonEnglishHHPct FLOAT)')

AnalysisException: "Hive support is required to CREATE Hive TABLE (AS SELECT);;\n'CreateTable `hh_df_from_sql`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore\n"